In [1]:
// spark 初始化，为后续sql的运行准备环境
import $ivy.`com.alibaba:fastjson:1.2.70`
import com.alibaba.fastjson.{JSON, JSONObject}
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}
var user = sys.env.get("JUPYTERHUB_USER").get
val conf = {
   new SparkConf()
   .setAppName(s"Online-Feature-For-Bid-${user}")
   .setMaster("yarn")
   .set("spark.yarn.queue", "root.zw03.hadoop-waimaiadrd.etl") // spark 的队列，可以根据自己的需要设置
   .set("spark.yarn.archive", "viewfs://hadoop-meituan/user/hadoop-hdp/notebook/spark-jars/spark_libs.zip")
   .set("spark.yarn.stagingDir", "viewfs://hadoop-meituan/ghnn03/sparkstaging")
   .set("spark.executor.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.driver.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
   .set("spark.executorEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.yarn.appMasterEnv.JAVA_HOME", "'/usr/local/java18'")
   .set("spark.sql.hive.metastorePartitionPruning", "true")
   .set("spark.shuffle.service.enabled", "true")
   .set("spark.dynamicAllocation.enabled", "true")
   .set("spark.dynamicAllocation.minExecutors", "600") //spark的执行Executors
   .set("spark.dynamicAllocation.maxExecutors","1000")
   .set("spark.executor.memory","16G")
   .set("spark.driver.memory","7G")
   .set("spark.input.dir.recursive","true")
   .set("spark.sql.ignore.existed.function.enable","true")
   .set("spark.yarn.maxAppAttempts", "2")
   .set("spark.noResourceMaxInterval","86400s")
   .set("spark.scheduler.mode", "FAIR")
   .set("spark.executor.cores","8")
   .set("spark.yarn.executor.memoryOverhead","32024")
   .set("spark.yarn.driver.memoryOverhead","32024")
   .set("spark.hadoop.hive.mt.renew.token.enable", "true")
}
val spark = {
 NotebookSparkSession
   .builder()
   .config(conf)
   .enableHiveSupport.getOrCreate
}
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
spark.sparkContext.applicationId

var logo="AFO_RL"
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
import java.text.SimpleDateFormat
import java.util.Date

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#noProviders for further details.
SLF4J: Class path contains SLF4J bindings targeting slf4j-api versions prior to 1.8.
SLF4J: Ignoring binding found at [jar:file:/conda/envs/notebook/share/jupyter/kernels/scala211/pack/lib/slf4j-nop-1.7.26.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#ignoredBindings for an explanation.


Adding Hadoop conf dir /opt/meituan/hadoop/etc/hadoop to classpath
Adding Hive conf dir /opt/meituan/spark-2.2/conf to classpath
Creating SparkSession


log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Spark UI

import $ivy.$                            

import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.{SparkConf, SparkContext}

user: String = "wanghao251"
conf: SparkConf = org.apache.spark.SparkConf@5231567e
spark: SparkSession = org.apache.spark.sql.SparkSession@65135413
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

res0_15: String = "application_1673505517722_414465"
logo: String = "AFO_RL"
import java.text.SimpleDateFormat

import java.util.Calendar

import ammonite.ops.ImplicitWd._

import ammonite.ops._

import sys.process._

import java.io._

import java.text.SimpleDateFormat

import java.util.Date

In [2]:
// 通用udf注册
import scala.collection.mutable.ArrayBuffer
import scala.math
import scala.util.Random;
import java.lang.Math
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}

import scala.collection.immutable.Map
import scala.collection.mutable

def JSONObj2Map(jsonObj: JSONObject): Map[String, String] = {
    var map = mutable.Map[String, String]()
    if(!jsonObj.isEmpty){
        val itr: java.util.Iterator[String] = jsonObj.keySet().iterator()
        while (itr.hasNext) {
            val key = itr.next()
            map += ((key, jsonObj.getString(key)))
        }
    }
    map.toMap
}

import org.apache.hadoop.hive.ql.exec.UDF
import spark.implicits._
val findIndex2 =  (strQuote:String) => {
    val poi = strQuote.split("\073").last
    val ind = strQuote.split("\073").zipWithIndex.filter(_._1 == poi).map(_._2).min
    ind.toString
}
spark.sqlContext.udf.register("findIndex2", findIndex2)
spark.udf.register("findIndex2", findIndex2)

def getDate(delta:Int, current_day:String = "today"):String = {
    val sdf:SimpleDateFormat = new SimpleDateFormat("yyyyMMdd")
    val cal:Calendar = Calendar.getInstance()
    if ( !current_day.equals("today")){
      val dt = sdf.parse(current_day)
      cal.setTime(dt)
    }
    cal.add(Calendar.DATE,delta)
    val day = sdf.format(cal.getTime)
    day
}
def getTimeSuffix():String={
    val dateFormat = new SimpleDateFormat("yyyyMMdd")
    val cla = Calendar.getInstance()
    cla.setTimeInMillis(System.currentTimeMillis())
    val table_suffix = dateFormat.format(cla.getTime)
    table_suffix
}
def dropTable(sqlContext:SQLContext, tableName:String, is_execute_sql:Boolean):Unit={
    printCurTime()
    var sql = s"Drop table if exists ${tableName}"
    if(is_execute_sql){
        println(s"执行 \n ${sql}")
        sqlContext.sql(sql)
    }
    printCurTime()
}

def executeSQL(sqlContext:SQLContext, sql:String, is_execute_sql:Boolean):DataFrame={
    printCurTime()
    println(s"执行 \n ${sql}")
    var result_ = sqlContext.sql(sql)
    printCurTime()
    result_
}

def genDateList(start: String, end: String): List[String] = {
    import java.util.Date
    import org.joda.time.{DateTime, Instant, Days}
    import org.joda.time.format.{DateTimeFormat, DateTimeFormatter}

    val DATE_KEY_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")
    val DATE_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")

    val startDate = DATE_KEY_FORMAT.parseDateTime(start).toDateTime
    val endDate = DATE_KEY_FORMAT.parseDateTime(end).toDateTime

    val daysCount = Days.daysBetween(startDate, endDate).getDays() + 1
    (0 until daysCount).map(startDate.plusDays(_)).map(d => d.toString(DATE_FORMAT)).toList
  }



def printCurTime():Unit={
    println(new SimpleDateFormat("yyyy-MM-dd HH:mm:ss").format(new Date))
}


def showTopNFromSQL(sql:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,sql,true)
    result.show(n,false)
    printCurTime()
}
def executeSQLLazy(sql:String):DataFrame={
    executeSQL(spark.sqlContext,sql,true)
}
def showTopNFromTableName(tableName:String, n:Int):Unit={
    printCurTime()
    var result = executeSQL(spark.sqlContext,s"""select * from ${tableName} limit ${n}""",true)
    result.show(n)
    printCurTime()
}
def executeShell(shell:String):Unit={
    printCurTime()
    println("----------------The content of shell is: -------------------")
    println(shell)
    println("----------------Start execution....-------------------")
    var file_name=System.currentTimeMillis()
    val writer = new PrintWriter(new File(s"/opt/meituan/dolphinfs_${user}/${file_name}.sh" ))
    writer.write(shell)
    writer.close()
    //启动脚本
    s"chmod +777 /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"sh /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    s"rm -r /opt/meituan/dolphinfs_${user}/${file_name}.sh".!
    println("----------------End-------------------")
    printCurTime()
}

def getFeatureMap(x:((String,String))):String={
    var x_1 = x._1
    var x_2 = x._2.replace("Type","")
    if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
    }
    var x_3 = s"""${x_1} ${x_2}"""
    x_3
}
// 传入sql，返回结果表
def createTableFromSQLWithName(sql:String,tableName:String,database:String):Unit={
    var rowData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var finalTableName=database+"."+tableName
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    
    
    //var featureForCreateTable=table_scheme.map{x=> s"""${x._1} ${x._2.stripSuffix("Type")}""".replace("Type","")}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    executeSQLLazy(s"insert OVERWRITE table ${finalTableName} select ${columnsName} from ${tmpTableName}")
}

def createTableFromSQL(sql:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,"tmp")
    "tmp."+finalTableName
}
def createTableFromSQL(sql:String,database:String):String={
    var finalTableName=s"${logo}_${System.currentTimeMillis()}"
    createTableFromSQLWithName(sql,finalTableName,database)
    database+"."+finalTableName
}

def generateTrajectory(tableName:String, groupColumns:Array[String], rankColumns:Array[String], joinColumns:Array[String],database:String):String={
    
    println(tableName.split("."))
    var finalTableName=database+"."+tableName.split("\\.")(1)+"_trajectory"
    
    var table_scheme = executeSQLLazy(s"""select * from ${tableName} limit 1""").dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var groupColumnString=groupColumns.mkString(",")
    var rankColumnString = rankColumns.mkString(",")
    var joinConditionString = joinColumns.map(x=> s"t1.${x}=t2.${x} ").mkString("\n and ")
    var selectT1columnsNameString = table_scheme.map{x=> s"t1.${x._1}"}.mkString(",")
    var selectT2columnsNameString = table_scheme.map{x=> s"t2.${x._1} as next_${x._1}"}.mkString(",")
    //var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",") + ",rn int,"+table_scheme.map(x=> {
        var x_1 = x._1
        var x_2 = x._2.replace("Type","")
        if (x_2.contains("Array")){
            var x_type = x_2.split('(')(1).split(",")(0)
            x_2  = s"""Array<${x_type}>"""
        }
        
        s"""next_${x_1} ${x_2}"""}
                                              ).mkString(",")+",next_rn int"
    var allColumnString=table_scheme.map{x=> s"${x._1}"}.mkString(",")+",rn,"+table_scheme.map{x=> s"next_${x._1}"}.mkString(",")+",next_rn"
    var sql=s"""
        select ${selectT1columnsNameString},t1.rn,${selectT2columnsNameString},t2.rn as next_rn
        from(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t1
        left join(
            select ${columnsName}, RANK() OVER(PARTITION BY ${groupColumnString} ORDER BY ${rankColumnString} asc) AS rn 
            from ${tableName}
        )t2
        on t1.rn+1 = t2.rn
        and ${joinConditionString}
        """
    var trajectoryData=executeSQLLazy(sql)
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable}) STORED AS ORC""")
    trajectoryData.createOrReplaceTempView(tmpTableName)
    spark.sql(s"insert OVERWRITE table ${finalTableName} select ${allColumnString} from ${tmpTableName}")
    finalTableName
}
def createPTTableFromTable(tableName:String,targetTableName:String,database:String):String={
    var finalTableName=targetTableName
    var rowData=executeSQLLazy(s"select * from ${tableName} limit 1")
    var tmpTableName=s"${database}.${logo}_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt='${date_s}') select ${columnsName} from ${tmpTableName}")
    finalTableName
}

def createPTTableFromTableWithSpace(tableName:String,database:String):String={
    var finalTableName=database+s".${logo}_${System.currentTimeMillis()}"
    var rowData=executeSQLLazy(s"select * from ${tableName}")
    var tmpTableName=s"tmp_${System.currentTimeMillis()}"
    var table_scheme = rowData.dtypes
    var columnsName = table_scheme.map{x=> x._1}.mkString(",")
    var featureForCreateTable=table_scheme.map(getFeatureMap).mkString(",")
    executeSQLLazy(s"""drop table if exists ${finalTableName}""")
    executeSQLLazy(s"""create table if not exists ${finalTableName}(${featureForCreateTable})  PARTITIONED BY (pt string) STORED AS ORC""")
    rowData.createOrReplaceTempView(tmpTableName)
    var dt = "today"
    var date_s=getDate(-1,dt)
    spark.sql(s"INSERT OVERWRITE TABLE ${finalTableName} PARTITION (pt=${date_s}) select ${columnsName} from ${tmpTableName}")
    finalTableName
}

import scala.collection.mutable.ArrayBuffer

import scala.math

import scala.util.Random;

import java.lang.Math

import com.alibaba.fastjson.{JSON, JSONObject}


import org.apache.spark.sql.{DataFrame, Row}

import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}


import scala.collection.immutable.Map

import scala.collection.mutable


defined function JSONObj2Map
import org.apache.hadoop.hive.ql.exec.UDF

import spark.implicits._

findIndex2: String => String = <function1>
res1_13: expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  StringType,
  Some(List(StringType))
)
res1_14: expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  StringType,
  Some(List(StringType))
)
defined function getDate
defined function getTimeSuffix
defined function dropTable
defined function executeSQL
defined function genDateList
defined function printCurTime
defined function showTopNFromSQL
def

In [117]:
showTopNFromSQL("""
select count(*) as count,floor(precisebid[0]/(rawbid[0])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=1 group by k_floor order by k_floor
""",100)

2023-01-04 14:32:24
2023-01-04 14:32:24
执行 
 
select count(*) as count,floor(precisebid[0]/(rawbid[0])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=1 group by k_floor order by k_floor

2023-01-04 14:32:24


show at cmd85.sc:96

2000 / 2000

[Stage 60:===================================================>(1999 + 1) / 2000]

show at cmd85.sc:96

200 / 200

+---------+-------+
|count    |k_floor|
+---------+-------+
|18929    |null   |
|11876    |1      |
|80218    |2      |
|226170   |3      |
|533571   |4      |
|17486653 |5      |
|35425524 |6      |
|59264453 |7      |
|73286068 |8      |
|63161763 |9      |
|199488519|10     |
|20475846 |11     |
|3736364  |12     |
|1276533  |13     |
|513418   |14     |
|429530   |15     |
|65712    |16     |
|33857    |17     |
|25879    |18     |
|18183    |19     |
|23980    |20     |
|13947    |21     |
|11741    |22     |
|8645     |23     |
|6420     |24     |
|9316     |25     |
|5036     |26     |
|4500     |27     |
|3472     |28     |
|2411     |29     |
|5104     |30     |
|1644     |31     |
|1421     |32     |
|1329     |33     |
|1667     |34     |
|1307     |35     |
|803      |36     |
|787      |37     |
|628      |38     |
|464      |39     |
|1090     |40     |
|393      |41     |
|373      |42     |
|360      |43     |
|287      |44     |
|481      |45     |
|237      |46     |


In [118]:
showTopNFromSQL("""
select count(*) as count,floor(precisebid[1]/(rawbid[1]*histctr[1])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=2 group by k_floor order by k_floor
""",100)

2023-01-04 14:33:31
2023-01-04 14:33:31
执行 
 
select count(*) as count,floor(precisebid[1]/(rawbid[1]*histctr[1])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=2 group by k_floor order by k_floor

2023-01-04 14:33:31


show at cmd85.sc:96

2000 / 2000

[Stage 62:===================================================>(1999 + 1) / 2000]

show at cmd85.sc:96

200 / 200

+-------+-------+
|count  |k_floor|
+-------+-------+
|540    |null   |
|3255   |1      |
|4665   |2      |
|187589 |3      |
|841000 |4      |
|532289 |5      |
|555831 |6      |
|657006 |7      |
|1088045|8      |
|764799 |9      |
|714362 |10     |
|819801 |11     |
|720614 |12     |
|896800 |13     |
|689518 |14     |
|612767 |15     |
|528507 |16     |
|655561 |17     |
|417557 |18     |
|311739 |19     |
|266699 |20     |
|247411 |21     |
|188672 |22     |
|308782 |23     |
|131234 |24     |
|110520 |25     |
|103837 |26     |
|93009  |27     |
|80699  |28     |
|67384  |29     |
|52663  |30     |
|48937  |31     |
|44411  |32     |
|42274  |33     |
|242160 |34     |
|36410  |35     |
|35237  |36     |
|34826  |37     |
|23573  |38     |
|15751  |39     |
|17743  |40     |
|16348  |41     |
|18527  |42     |
|18008  |43     |
|23331  |44     |
|14963  |45     |
|13387  |46     |
|12723  |47     |
|14548  |48     |
|10902  |49     |
|10719  |50     |
|12407  |51     |
|11213  |5

In [119]:
showTopNFromSQL("""
select count(*) as count,floor(precisebid[2]/(rawbid[2])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=3 group by k_floor order by k_floor
""",100)

2023-01-04 14:34:24
2023-01-04 14:34:24
执行 
 
select count(*) as count,floor(precisebid[2]/(rawbid[2])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=3 group by k_floor order by k_floor

2023-01-04 14:34:24


show at cmd85.sc:96

2000 / 2000

[Stage 64:===================================================>(1999 + 1) / 2000]

show at cmd85.sc:96

200 / 200

+--------+-------+
|count   |k_floor|
+--------+-------+
|3200    |null   |
|44      |0      |
|2954    |1      |
|20169   |2      |
|31360   |3      |
|52175   |4      |
|506205  |5      |
|1314338 |6      |
|2668887 |7      |
|9886612 |8      |
|22298059|9      |
|49258535|10     |
|17029690|11     |
|1197049 |12     |
|367787  |13     |
|123877  |14     |
|1624695 |15     |
|21460   |16     |
|13107   |17     |
|12569   |18     |
|8096    |19     |
|8762    |20     |
|7731    |21     |
|5390    |22     |
|4603    |23     |
|2658    |24     |
|3414    |25     |
|2516    |26     |
|3832    |27     |
|3205    |28     |
|1189    |29     |
|2044    |30     |
|641     |31     |
|642     |32     |
|773     |33     |
|1350    |34     |
|516     |35     |
|463     |36     |
|1171    |37     |
|779     |38     |
|146     |39     |
|986     |40     |
|61      |41     |
|353     |42     |
|130     |43     |
|82      |44     |
|603     |45     |
|234     |46     |
|176     |47     |
|93      |48

In [120]:
showTopNFromSQL("""
select count(*) as count,floor(precisebid[3]/(rawbid[3]*histctr[3])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=4 group by k_floor order by k_floor
""",100)

2023-01-04 14:34:53
2023-01-04 14:34:53
执行 
 
select count(*) as count,floor(precisebid[3]/(rawbid[3]*histctr[3])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=4 group by k_floor order by k_floor

2023-01-04 14:34:53


show at cmd85.sc:96

2000 / 2000

[Stage 66:===================================================>(1998 + 2) / 2000]

show at cmd85.sc:96

200 / 200

+-------+-------+
|count  |k_floor|
+-------+-------+
|252    |null   |
|4124656|0      |
|6839040|1      |
|3136659|2      |
|2468605|3      |
|3008430|4      |
|2662836|5      |
|1826519|6      |
|1980642|7      |
|1392989|8      |
|694341 |9      |
|539705 |10     |
|681835 |11     |
|489399 |12     |
|193996 |13     |
|607998 |14     |
|114464 |15     |
|488445 |16     |
|60339  |17     |
|597723 |18     |
|123915 |19     |
|47860  |20     |
|76994  |21     |
|597541 |22     |
|22437  |23     |
|88052  |24     |
|36780  |25     |
|33863  |26     |
|30239  |27     |
|582064 |28     |
|18433  |29     |
|31390  |30     |
|12039  |31     |
|57227  |32     |
|12645  |33     |
|12710  |34     |
|8592   |35     |
|34594  |36     |
|424937 |37     |
|16096  |38     |
|34870  |39     |
|22519  |40     |
|10633  |41     |
|14063  |42     |
|23398  |43     |
|12855  |44     |
|31101  |45     |
|12395  |46     |
|14768  |47     |
|27165  |48     |
|57778  |49     |
|11045  |50     |
|10734  |5

In [122]:
showTopNFromSQL("""
select count(*) as count,floor(precisebid[0]/(rawbid[0])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=1 and is_expose[0]=1 group by k_floor order by k_floor
""",100)

2023-01-04 14:47:31
2023-01-04 14:47:31
执行 
 
select count(*) as count,floor(precisebid[0]/(rawbid[0])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=1 and is_expose[0]=1 group by k_floor order by k_floor

2023-01-04 14:47:31


show at cmd85.sc:96

2000 / 2000

[Stage 68:===================================================>(1999 + 1) / 2000]

show at cmd85.sc:96

200 / 200

+--------+-------+
|count   |k_floor|
+--------+-------+
|1539    |null   |
|806     |1      |
|2568    |2      |
|5517    |3      |
|12198   |4      |
|447192  |5      |
|1083334 |6      |
|2927757 |7      |
|5988439 |8      |
|8012245 |9      |
|16597162|10     |
|4686708 |11     |
|1222537 |12     |
|439833  |13     |
|175146  |14     |
|120291  |15     |
|3099    |16     |
|2137    |17     |
|1684    |18     |
|1180    |19     |
|1382    |20     |
|881     |21     |
|724     |22     |
|520     |23     |
|348     |24     |
|416     |25     |
|319     |26     |
|327     |27     |
|209     |28     |
|164     |29     |
|275     |30     |
|138     |31     |
|132     |32     |
|93      |33     |
|121     |34     |
|110     |35     |
|74      |36     |
|82      |37     |
|73      |38     |
|41      |39     |
|57      |40     |
|41      |41     |
|40      |42     |
|30      |43     |
|27      |44     |
|39      |45     |
|22      |46     |
|16      |47     |
|23      |48     |
|14      |49

In [123]:
showTopNFromSQL("""
select count(*) as count,floor(precisebid[1]/(rawbid[1]*histctr[1])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=2 and  is_expose[1]=1 group by k_floor order by k_floor
""",100)

2023-01-04 14:49:47
2023-01-04 14:49:47
执行 
 
select count(*) as count,floor(precisebid[1]/(rawbid[1]*histctr[1])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=2 and  is_expose[1]=1 group by k_floor order by k_floor

2023-01-04 14:49:47


show at cmd85.sc:96

2000 / 2000

[Stage 70:===================================================>(1999 + 1) / 2000]

show at cmd85.sc:96

200 / 200

+------+-------+
|count |k_floor|
+------+-------+
|98    |null   |
|2015  |1      |
|1883  |2      |
|63877 |3      |
|181423|4      |
|163568|5      |
|183617|6      |
|203771|7      |
|289394|8      |
|217869|9      |
|203939|10     |
|238107|11     |
|216294|12     |
|248761|13     |
|211060|14     |
|195247|15     |
|169599|16     |
|220170|17     |
|136310|18     |
|105174|19     |
|91630 |20     |
|83268 |21     |
|61965 |22     |
|86192 |23     |
|40789 |24     |
|34506 |25     |
|31183 |26     |
|27258 |27     |
|21804 |28     |
|15883 |29     |
|13059 |30     |
|12043 |31     |
|9222  |32     |
|9824  |33     |
|54803 |34     |
|7567  |35     |
|6095  |36     |
|7668  |37     |
|4801  |38     |
|3202  |39     |
|3289  |40     |
|3110  |41     |
|3643  |42     |
|3088  |43     |
|3335  |44     |
|3978  |45     |
|2741  |46     |
|2448  |47     |
|2143  |48     |
|1745  |49     |
|2266  |50     |
|2172  |51     |
|3041  |52     |
|2049  |53     |
|3823  |54     |
|2055  |55    

In [124]:
showTopNFromSQL("""
select count(*) as count,floor(precisebid[2]/(rawbid[2])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=3 and is_expose[2]=1 group by k_floor order by k_floor
""",100)

2023-01-04 14:52:34
2023-01-04 14:52:34
执行 
 
select count(*) as count,floor(precisebid[2]/(rawbid[2])*100/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=3 and is_expose[2]=1 group by k_floor order by k_floor

2023-01-04 14:52:34


show at cmd85.sc:96

2000 / 2000

[Stage 72:===================================================>(1999 + 1) / 2000]

show at cmd85.sc:96

200 / 200

+-------+-------+
|count  |k_floor|
+-------+-------+
|356    |null   |
|1      |0      |
|71     |1      |
|349    |2      |
|581    |3      |
|936    |4      |
|34682  |5      |
|112777 |6      |
|222217 |7      |
|567807 |8      |
|1150900|9      |
|2295016|10     |
|796478 |11     |
|68072  |12     |
|20214  |13     |
|7340   |14     |
|34309  |15     |
|409    |16     |
|304    |17     |
|236    |18     |
|164    |19     |
|185    |20     |
|114    |21     |
|105    |22     |
|106    |23     |
|31     |24     |
|51     |25     |
|56     |26     |
|64     |27     |
|50     |28     |
|22     |29     |
|30     |30     |
|21     |31     |
|14     |32     |
|12     |33     |
|24     |34     |
|16     |35     |
|11     |36     |
|12     |37     |
|17     |38     |
|3      |39     |
|24     |40     |
|2      |41     |
|3      |42     |
|2      |43     |
|4      |45     |
|2      |46     |
|2      |47     |
|2      |48     |
|3      |49     |
|15     |50     |
|1      |52     |
|3      |5

In [14]:
showTopNFromSQL("""
select count(*) as count,floor(precisebid[3]/(rawbid[3]*histctr[3])*1000/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=4 group by k_floor order by count desc
""",100)

2023-01-05 14:55:08
2023-01-05 14:55:08
执行 
 
select count(*) as count,floor(precisebid[3]/(rawbid[3]*histctr[3])*1000/10) as k_floor
from tmp.AFO_RL_1672813547056 where prod=4 and is_expose[3]=1 group by k_floor order by count desc

2023-01-05 14:55:08


show at cmd1.sc:96

2001 / 2000

[Stage 43:===================================================>(1999 + 1) / 2000]

show at cmd1.sc:96

200 / 200

+-------+-------+
|count  |k_floor|
+-------+-------+
|1187611|70     |
|894916 |62     |
|847542 |56     |
|602540 |10     |
|589467 |11     |
|587067 |9      |
|481896 |80     |
|479065 |8      |
|453446 |7      |
|438077 |47     |
|434034 |13     |
|428474 |51     |
|428198 |12     |
|367304 |14     |
|344144 |6      |
|341202 |15     |
|338962 |43     |
|330974 |16     |
|251371 |40     |
|246165 |18     |
|218993 |37     |
|205216 |17     |
|184440 |21     |
|183335 |5      |
|174931 |20     |
|158506 |19     |
|148366 |25     |
|140002 |113    |
|136917 |22     |
|126026 |23     |
|125418 |94     |
|120562 |33     |
|117085 |24     |
|116173 |48     |
|109230 |566    |
|101860 |30     |
|101421 |283    |
|97968  |36     |
|95622  |35     |
|92939  |27     |
|90940  |377    |
|88888  |26     |
|88069  |226    |
|86386  |32     |
|85089  |34     |
|83226  |188    |
|81455  |28     |
|80155  |31     |
|77817  |85     |
|77276  |42     |
|76218  |53     |
|75596  |141    |
|68927  |4

In [130]:
showTopNFromSQL("""
select prod,count(*) from tmp.AFO_RL_1672813547056 where is_expose[0]+is_expose[1]+is_expose[2]+is_expose[3]>0 group by prod
""",5)

2023-01-04 15:00:58
2023-01-04 15:00:58
执行 
 
select prod,count(*) from tmp.AFO_RL_1672813547056 where is_expose[0]+is_expose[1]+is_expose[2]+is_expose[3]>0 group by prod

2023-01-04 15:00:58


show at cmd85.sc:96

2000 / 2000

[Stage 96:===================================================>(1999 + 1) / 2000]

show at cmd85.sc:96

...

show at cmd85.sc:96

100 / 100

show at cmd85.sc:96

75 / 75

+----+--------+
|prod|count(1)|
+----+--------+
|2.0 |5004717 |
|3.0 |5314250 |
|1.0 |41738329|
|4.0 |16709173|
+----+--------+

2023-01-04 15:02:09


In [131]:
showTopNFromSQL("""
select prod,count(*) from tmp.AFO_RL_1672813547056 where is_click[0]+is_click[1]+is_click[2]+is_click[3]>0 group by prod
""",5)

2023-01-04 15:03:14
2023-01-04 15:03:14
执行 
 
select prod,count(*) from tmp.AFO_RL_1672813547056 where is_click[0]+is_click[1]+is_click[2]+is_click[3]>0 group by prod

2023-01-04 15:03:14


show at cmd85.sc:96

2020 / 2000

[Stage 106:==================================================>(1999 + 1) / 2000]

show at cmd85.sc:96

...

show at cmd85.sc:96

100 / 100

show at cmd85.sc:96

75 / 75

+----+--------+
|prod|count(1)|
+----+--------+
|2.0 |218528  |
|3.0 |473308  |
|1.0 |1913887 |
|4.0 |152695  |
+----+--------+

2023-01-04 15:04:59


In [139]:
showTopNFromSQL("""
select count(*),poi_id from mart_waimaiad.AFO_RL_1672813547056_trajectory where prod=1 group by poi_id
""",100)

2023-01-04 15:25:17
2023-01-04 15:25:17
执行 
 
select count(*),poi_id from mart_waimaiad.AFO_RL_1672813547056_trajectory where prod=1 group by poi_id

2023-01-04 15:25:17


show at cmd85.sc:96

200 / 200

[Stage 122:====================================================>(199 + 1) / 200]

show at cmd85.sc:96

1 / 1

+--------+--------+
|count(1)|poi_id  |
+--------+--------+
|9       |15853408|
|21920   |15925942|
|11864   |16094485|
|2139    |13372477|
|960     |7254668 |
|11223   |12718988|
|6111    |724689  |
|3683    |11903180|
|6044    |9017896 |
|401     |15351277|
|821     |15343960|
|9       |16211512|
|268     |14895457|
|5313    |15211705|
|161     |16482621|
|9239    |15333032|
|1908    |16500638|
|786     |1670932 |
|12804   |15945079|
|5630    |16464547|
|23278   |16474132|
|31720   |15373536|
|22273   |11714979|
|166     |15763606|
|1277    |1665826 |
|546     |14290419|
|27121   |9508822 |
|579     |12184974|
|3937    |15911041|
|229     |9298753 |
|795     |15033866|
|392     |9314032 |
|664     |13456313|
|1696    |15674452|
|15557   |5611751 |
|6710    |14685358|
|3766    |9404777 |
|6451    |7029731 |
|3271    |9766015 |
|16628   |14551500|
|5114    |14982762|
|114381  |16097148|
|10840   |16255361|
|600     |5212164 |
|14377   |12360701|
|2190    |5937418 |
|14285   |15858641|


In [9]:
showTopNFromSQL("""
select mean(count) from(
select count(*) as count from mart_waimaiad.dsa_model_fp_os_rl_log_wh where dt=20230101 and is_expose=1 group by poi_id )
""",10)

2023-01-05 14:26:41
2023-01-05 14:26:41
执行 
 
select mean(count) from(
select count(*) as count from mart_waimaiad.dsa_model_fp_os_rl_log_wh where dt=20230101 and is_expose=1 group by poi_id )

2023-01-05 14:26:42


show at cmd1.sc:96

2000 / 2000

[Stage 25:===================================================>(1996 + 4) / 2000]

show at cmd1.sc:96

200 / 200

[Stage 26:=====================================================>(199 + 1) / 200]

show at cmd1.sc:96

1 / 1

+------------------+
|avg(count)        |
+------------------+
|1463.2002455216527|
+------------------+

2023-01-05 14:29:10


In [13]:
showTopNFromSQL("""
select mean(count),prod from(
select count(*) as count, prod from mart_waimaiad.AFO_RL_1672894959448 where dt=20230101 and is_expose[0]+is_expose[1]+is_expose[2]+is_expose[3]=1 group by poi_id,prod) group by prod
""",10)

2023-01-05 14:35:35
2023-01-05 14:35:35
执行 
 
select mean(count),prod from(
select count(*) as count, prod from mart_waimaiad.AFO_RL_1672894959448 where dt=20230101 and is_expose[0]+is_expose[1]+is_expose[2]+is_expose[3]=1 group by poi_id,prod) group by prod

2023-01-05 14:35:35


show at cmd1.sc:96

200 / 200

[Stage 28:=====================================================>(199 + 1) / 200]

show at cmd1.sc:96

...

show at cmd1.sc:96

100 / 100

show at cmd1.sc:96

75 / 75

+----------+----+
|avg(count)|prod|
+----------+----+
+----------+----+

2023-01-05 14:37:32


In [15]:
showTopNFromSQL(s"""
select count(*),is_expose from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230102 group by is_expose
""",30)

2023-01-05 14:57:38
2023-01-05 14:57:38
执行 
 
select count(*),is_expose from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230102 group by is_expose

2023-01-05 14:57:38


show at cmd1.sc:96

2000 / 2000

[Stage 45:===================================================>(1999 + 1) / 2000]

show at cmd1.sc:96

...

show at cmd1.sc:96

100 / 100

show at cmd1.sc:96

75 / 75

+---------+---------+
|count(1) |is_expose|
+---------+---------+
|585559698|0        |
|70293400 |1        |
+---------+---------+

2023-01-05 14:58:38


In [11]:
showTopNFromSQL("""
select count(*),dt from mart_waimaiad.dsa_model_fp_os_rl_log_wh group by dt 

""",10)

2023-01-07 14:29:15
2023-01-07 14:29:15
执行 
 
select count(*),dt from mart_waimaiad.dsa_model_fp_os_rl_log_wh group by dt 


2023-01-07 14:29:16


show at cmd9.sc:96

10000 / 10000

[Stage 0:===================================================>(9999 + 1) / 10000]

show at cmd9.sc:96

...

show at cmd9.sc:96

100 / 100

show at cmd9.sc:96

75 / 75

+---------+--------+
|count(1) |dt      |
+---------+--------+
|875798641|20230101|
|917737399|20230106|
|893610841|20230104|
|863453644|20230103|
|857434405|20230102|
+---------+--------+

2023-01-07 14:31:56


In [25]:
showTopNFromSQL("""
select action[0],count(*) from mart_waimaiad.AFO_RL_1672389719048  group by action[0] order by action[0]
""",30)

2023-01-07 22:35:16
2023-01-07 22:35:16
执行 
 
select action[0],count(*) from mart_waimaiad.AFO_RL_1672389719048  group by action[0] order by action[0]

2023-01-07 22:35:17


show at cmd9.sc:96

200 / 200

[Stage 39:=====================================================>(199 + 1) / 200]

show at cmd9.sc:96

200 / 200

+---------+--------+
|action[0]|count(1)|
+---------+--------+
|0        |10437   |
|3        |7       |
|4        |5357    |
|6        |6       |
|8        |4291    |
|9        |2       |
|12       |15354   |
|16       |375     |
|18       |2       |
|20       |374     |
|24       |1049    |
|28       |485     |
|32       |411     |
|36       |973     |
|39       |2       |
|40       |525     |
|42       |1       |
|44       |312     |
|48       |478     |
|52       |574     |
|56       |280     |
|57       |1       |
|60       |1466    |
|64       |461     |
|68       |393     |
|72       |644     |
|76       |1854    |
|80       |13337   |
+---------+--------+

2023-01-07 22:35:31


In [26]:
showTopNFromSQL("""
select * from mart_waimaiad.AFO_RL_1672389719048 where action[0]=0
""",5)

2023-01-07 22:36:20
2023-01-07 22:36:20
执行 
 
select * from mart_waimaiad.AFO_RL_1672389719048 where action[0]=0

2023-01-07 22:36:20


show at cmd9.sc:96

1 / 1

+--------+--------+-------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------+------------------------+--------------------------+-------------------------------------------------------------------+--------------------------------------------------+--------------------+--------------------+-------------------------+----------------------+--------------------------------------------------------------------------------+---------------------+-------------------------------------------------+--------------------+-----------------------------------------------------------------------------+------------------------+---------------------+-------------------------------------------------------------------+----------------------------------+------------------------------------------------------------------+---------------------+---------------------+---------------------------+----

In [14]:
showTopNFromSQL("""
select count(*) from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where qsbid<10 and prod=2
""",10)

2023-01-09 10:48:16
2023-01-09 10:48:16
执行 
 
select count(*) from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where qsbid<10 and prod=2

2023-01-09 10:48:16


show at cmd1.sc:96

2000 / 2000

[Stage 9:====================================================>(1999 + 1) / 2000]

show at cmd1.sc:96

1 / 1

+--------+
|count(1)|
+--------+
|7203153 |
+--------+

2023-01-09 10:50:12


In [22]:
showTopNFromSQL("""
select count(*) from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=2 and histctr=0.01
""",10)

2023-01-09 11:55:43
2023-01-09 11:55:43
执行 
 
select count(*) from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=2 and histctr=0.01

2023-01-09 11:55:43


show at cmd1.sc:96

2000 / 2000

[Stage 23:===================================================>(1999 + 1) / 2000]

show at cmd1.sc:96

1 / 1

+--------+
|count(1)|
+--------+
|0       |
+--------+

2023-01-09 12:00:03


In [23]:
showTopNFromSQL("""
select count(*) as count,floor(precisebid/(aimcpc*histctr)*100/10) as k_floor
from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=2 group by k_floor order by k_floor
""",100)

2023-01-09 12:06:04
2023-01-09 12:06:04
执行 
 
select count(*) as count,floor(precisebid/(aimcpc*histctr)*100/10) as k_floor
from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=2 group by k_floor order by k_floor

2023-01-09 12:06:04


show at cmd1.sc:96

2000 / 2000

[Stage 25:===================================================>(1999 + 1) / 2000]

show at cmd1.sc:96

200 / 200

+--------+-------+
|count   |k_floor|
+--------+-------+
|4469    |null   |
|175     |1      |
|57796   |2      |
|556368  |3      |
|3390595 |4      |
|5390872 |5      |
|8478716 |6      |
|6919138 |7      |
|6830638 |8      |
|7202625 |9      |
|4054253 |10     |
|7563042 |11     |
|4274823 |12     |
|9300779 |13     |
|3883209 |14     |
|4142505 |15     |
|3313717 |16     |
|11505428|17     |
|4255807 |18     |
|3008504 |19     |
|2887621 |20     |
|3073667 |21     |
|2241702 |22     |
|6754476 |23     |
|2313113 |24     |
|2029420 |25     |
|2082807 |26     |
|2110524 |27     |
|1671436 |28     |
|1632885 |29     |
|1295998 |30     |
|1198637 |31     |
|1172396 |32     |
|1060996 |33     |
|7149074 |34     |
|907891  |35     |
|1360766 |36     |
|1054586 |37     |
|704117  |38     |
|625908  |39     |
|580287  |40     |
|588335  |41     |
|561538  |42     |
|609357  |43     |
|601813  |44     |
|399022  |45     |
|405038  |46     |
|391520  |47     |
|423534  |48     |
|372481  |49

In [34]:
showTopNFromSQL("""
select poi_id,precisebid,aimcpc,histctr,aimcpc*histctr, precisebid/(aimcpc*histctr) as k_floor
from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=2
""",100)

2023-01-09 14:44:38
2023-01-09 14:44:38
执行 
 
select poi_id,precisebid,aimcpc,histctr,aimcpc*histctr, precisebid/(aimcpc*histctr) as k_floor
from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=2

2023-01-09 14:44:38


show at cmd1.sc:96

1 / 1

+--------+------------------+------+--------------------+--------------------------------------------------+-------------------+
|poi_id  |precisebid        |aimcpc|histctr             |(CAST(aimcpc AS DOUBLE) * CAST(histctr AS DOUBLE))|k_floor            |
+--------+------------------+------+--------------------+--------------------------------------------------+-------------------+
|15698859|3.0               |60.0  |0.04287802053479448 |2.572681232087669                                 |1.1660986066142256 |
|13254546|3.0               |70.0  |0.02491908570529908 |1.7443359993709355                                |1.7198521391990407 |
|16284287|3.4502963163405966|100.0 |0.04287802053479448 |4.2878020534794485                                |0.804677145378193  |
|11618263|3.0               |40.0  |0.04287802053479448 |1.7151208213917792                                |1.7491479099213385 |
|13050009|3.0               |200.0 |0.027698742392861114|5.539748478572223                       

In [30]:
showTopNFromSQL("""
select precisebid,aimcpc,histctr,aimcpc*histctr, precisebid/(aimcpc*histctr) as k_floor
from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=4 and is_click=1
""",100)

2023-01-09 14:28:42
2023-01-09 14:28:42
执行 
 
select precisebid,aimcpc,histctr,aimcpc*histctr, precisebid/(aimcpc*histctr) as k_floor
from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=4 and is_click=1

2023-01-09 14:28:42


show at cmd1.sc:96

1 / 1

show at cmd1.sc:96

1 / 1

+------------------+------+---------------------+--------------------------------------------------+-------------------+
|precisebid        |aimcpc|histctr              |(CAST(aimcpc AS DOUBLE) * CAST(histctr AS DOUBLE))|k_floor            |
+------------------+------+---------------------+--------------------------------------------------+-------------------+
|1.0               |90.0  |0.0616857023792755   |5.551713214134796                                 |0.18012457802286616|
|1.0               |80.0  |0.017660375257175495 |1.4128300205740396                                |0.707799229516439  |
|1.0               |80.0  |0.10464449880881227  |8.37155990470498                                  |0.11945205091801116|
|1.0               |110.0 |0.11958212381540423  |13.154033619694465                                |0.076022308358919  |
|1.0               |170.0 |0.017660375257175495 |3.002263793719834                                 |0.3330819903606772 |
|1.0               |120.0 |0.017

In [36]:
showTopNFromSQL("""
select count(*) as k_floor
from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=4 and precisebid<1
""",100)

2023-01-09 15:11:51
2023-01-09 15:11:51
执行 
 
select count(*) as k_floor
from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230107 where prod=4 and precisebid<1

2023-01-09 15:11:51


show at cmd1.sc:96

2000 / 2000

[Stage 45:===================================================>(1999 + 1) / 2000]

show at cmd1.sc:96

1 / 1

+-------+
|k_floor|
+-------+
|559103 |
+-------+

2023-01-09 15:13:49


In [5]:
showTopNFromSQL("""
select count(*) from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230116 where rlkAct='unknown'
""",10)

2023-01-17 11:31:53
2023-01-17 11:31:53
执行 
 
select count(*) from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230116 where rlkAct='unknown'

2023-01-17 11:31:53


show at cmd1.sc:96

2000 / 2000

[Stage 1:====================================================>(1999 + 1) / 2000]

show at cmd1.sc:96

1 / 1

+--------+
|count(1)|
+--------+
|92565857|
+--------+

2023-01-17 11:33:10


In [10]:
showTopNFromSQL("""
select max(k),min(k),prod,upperk,lowerk from 
(select cast(rlkAct as float) as k,prod,upperk,lowerk from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230116 where rlkAct!='unknown' ) group by prod,upperk,lowerk
""",10)

2023-01-17 11:47:58
2023-01-17 11:47:58
执行 
 
select max(k),min(k),prod,upperk,lowerk from 
(select cast(rlkAct as float) as k,prod,upperk,lowerk from tmp.dsa_model_fp_log_wanghao251_dsa_os_unify_model_df_20230116 where rlkAct!='unknown' ) group by prod,upperk,lowerk

2023-01-17 11:47:58


show at cmd1.sc:96

2000 / 2000

[Stage 25:===================================================>(1999 + 1) / 2000]

show at cmd1.sc:96

...

show at cmd1.sc:96

100 / 100

show at cmd1.sc:96

75 / 75

+------+------+----+------+------+
|max(k)|min(k)|prod|upperk|lowerk|
+------+------+----+------+------+
|30.0  |0.0   |4.0 |1.2   |0.5   |
|30.0  |0.0   |3.0 |1.2   |0.9   |
|30.0  |0.0   |2.0 |1.2   |0.5   |
|30.0  |0.0   |1.0 |1.2   |0.9   |
+------+------+----+------+------+

2023-01-17 11:49:12
